# Solar Power Generation Data Preprocessing and Feature Engineering

This notebook implements the data preprocessing pipeline for solar power generation prediction.
Target variable: generation(kWh)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer, KNNImputer
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Data directory
DATA_DIR = '/home/ubuntu/upload/'
OUTPUT_DIR = '/home/ubuntu/processed_data/'
os.makedirs(OUTPUT_DIR, exist_ok=True)

## 1. Data Loading Functions

In [ ]:
def load_power_generation_data():
    """Load and combine all power generation data"""
    all_files = glob.glob(os.path.join(DATA_DIR, '*.csv'))
    
    # Filter power generation files
    power_files = [f for f in all_files if not any(weather in f for weather in 
                   ['Temperature', 'Humidity', 'Irradiance', 'Wind', 'Visibility', 
                    'SeaLevelPressure', 'RelativeHumidity', 'Rainfall']) 
                   and not 'Inverter' in f]
    
    power_data_list = []
    
    for file in power_files:
        try:
            df = pd.read_csv(file)
            df['Time'] = pd.to_datetime(df['Time'])
            df['station'] = os.path.basename(file).replace('.csv', '')
            power_data_list.append(df)
            print(f"Loaded {os.path.basename(file)}: {df.shape[0]} records")
        except Exception as e:
            print(f"Error loading {file}: {e}")
    
    # Combine all power data
    combined_power = pd.concat(power_data_list, ignore_index=True)
    combined_power = combined_power.sort_values(['station', 'Time']).reset_index(drop=True)
    
    print(f"\nTotal power generation data: {combined_power.shape[0]:,} records")
    print(f"Stations: {combined_power['station'].nunique()}")
    print(f"Date range: {combined_power['Time'].min()} to {combined_power['Time'].max()}")
    
    return combined_power

def load_weather_data():
    """Load and combine all weather data"""
    all_files = glob.glob(os.path.join(DATA_DIR, '*.csv'))
    all_files.extend(glob.glob(os.path.join(DATA_DIR, '*.xlsx')))
    
    # Filter weather files
    weather_files = [f for f in all_files if any(weather in f for weather in 
                    ['Temperature', 'Humidity', 'Irradiance', 'Wind', 'Visibility', 
                     'SeaLevelPressure', 'RelativeHumidity', 'Rainfall'])]
    
    weather_data = {}
    
    for file in weather_files:
        weather_type = os.path.basename(file).split('_')[0]
        year = os.path.basename(file).split('_')[1].replace('.csv', '').replace('.xlsx', '')
        
        try:
            if file.endswith('.xlsx'):
                df = pd.read_excel(file)
            else:
                df = pd.read_csv(file)
            
            df['Time'] = pd.to_datetime(df['Time'])
            
            if weather_type not in weather_data:
                weather_data[weather_type] = []
            
            weather_data[weather_type].append(df)
            print(f"Loaded {weather_type}_{year}: {df.shape[0]} records")
            
        except Exception as e:
            print(f"Error loading {file}: {e}")
    
    # Combine weather data by type
    combined_weather = {}
    for weather_type, dfs in weather_data.items():
        combined_df = pd.concat(dfs, ignore_index=True)
        combined_df = combined_df.sort_values('Time').reset_index(drop=True)
        
        # Remove duplicates
        combined_df = combined_df.drop_duplicates(subset=['Time']).reset_index(drop=True)
        
        combined_weather[weather_type] = combined_df
        print(f"Combined {weather_type}: {combined_df.shape[0]:,} records")
    
    return combined_weather

## 2. Load All Data

In [ ]:
# Load power generation data
print("Loading power generation data...")
power_data = load_power_generation_data()

print("\n" + "="*50)
print("Loading weather data...")
weather_data = load_weather_data()

## 3. Data Quality Assessment and Cleaning

In [ ]:
# Assess power data quality
print("=== POWER DATA QUALITY ===")
print(f"Shape: {power_data.shape}")
print(f"Missing values:")
print(power_data.isnull().sum())
print(f"\nGeneration statistics:")
print(power_data['generation(kWh)'].describe())

# Check for negative values
negative_gen = (power_data['generation(kWh)'] < 0).sum()
print(f"\nNegative generation values: {negative_gen}")

# Check for outliers (values > 99th percentile)
gen_99th = power_data['generation(kWh)'].quantile(0.99)
outliers = (power_data['generation(kWh)'] > gen_99th).sum()
print(f"Values above 99th percentile ({gen_99th:.2f}): {outliers}")

In [ ]:
# Assess weather data quality
print("=== WEATHER DATA QUALITY ===")
for weather_type, df in weather_data.items():
    print(f"\n{weather_type}:")
    print(f"  Shape: {df.shape}")
    print(f"  Missing values: {df.isnull().sum().sum()}")
    print(f"  Date range: {df['Time'].min()} to {df['Time'].max()}")
    
    # Get numeric column
    numeric_col = [col for col in df.columns if col != 'Time'][0]
    print(f"  {numeric_col} range: {df[numeric_col].min():.2f} to {df[numeric_col].max():.2f}")

## 4. Weather Data Resampling and Aggregation

In [ ]:
def resample_weather_data(weather_data, target_freq='15T'):
    """Resample weather data to match power generation frequency"""
    resampled_weather = {}
    
    for weather_type, df in weather_data.items():
        # Get the numeric column name
        numeric_col = [col for col in df.columns if col != 'Time'][0]
        
        # Set Time as index for resampling
        df_indexed = df.set_index('Time')
        
        # Resample to target frequency (15 minutes) using mean
        resampled = df_indexed.resample(target_freq).agg({
            numeric_col: ['mean', 'min', 'max', 'std']
        })
        
        # Flatten column names
        resampled.columns = [f"{weather_type}_{stat}" for stat in ['mean', 'min', 'max', 'std']]
        
        # Reset index to get Time as column
        resampled = resampled.reset_index()
        
        resampled_weather[weather_type] = resampled
        print(f"Resampled {weather_type}: {resampled.shape[0]:,} records")
    
    return resampled_weather

# Resample weather data
print("Resampling weather data to 15-minute intervals...")
resampled_weather = resample_weather_data(weather_data)

## 5. Feature Engineering

In [ ]:
def create_time_features(df, time_col='Time'):
    """Create time-based features"""
    df = df.copy()
    
    # Basic time features
    df['year'] = df[time_col].dt.year
    df['month'] = df[time_col].dt.month
    df['day'] = df[time_col].dt.day
    df['hour'] = df[time_col].dt.hour
    df['minute'] = df[time_col].dt.minute
    df['day_of_week'] = df[time_col].dt.dayofweek
    df['day_of_year'] = df[time_col].dt.dayofyear
    df['week_of_year'] = df[time_col].dt.isocalendar().week
    
    # Cyclical features
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    # Solar position approximation
    df['solar_elevation'] = np.sin(2 * np.pi * (df['day_of_year'] - 81) / 365) * 23.45
    
    # Time of day categories
    df['is_daytime'] = ((df['hour'] >= 6) & (df['hour'] <= 18)).astype(int)
    df['is_peak_sun'] = ((df['hour'] >= 10) & (df['hour'] <= 14)).astype(int)
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    
    return df

def create_lag_features(df, target_col, lags=[1, 2, 3, 4, 24, 48, 96]):
    """Create lag features for time series"""
    df = df.copy()
    
    for lag in lags:
        df[f'{target_col}_lag_{lag}'] = df[target_col].shift(lag)
    
    return df

def create_rolling_features(df, target_col, windows=[4, 12, 24, 48, 96]):
    """Create rolling window features"""
    df = df.copy()
    
    for window in windows:
        df[f'{target_col}_rolling_mean_{window}'] = df[target_col].rolling(window=window, min_periods=1).mean()
        df[f'{target_col}_rolling_std_{window}'] = df[target_col].rolling(window=window, min_periods=1).std()
        df[f'{target_col}_rolling_max_{window}'] = df[target_col].rolling(window=window, min_periods=1).max()
        df[f'{target_col}_rolling_min_{window}'] = df[target_col].rolling(window=window, min_periods=1).min()
    
    return df

# Apply time features to power data
print("Creating time-based features...")
power_data_featured = create_time_features(power_data)
print(f"Added time features. New shape: {power_data_featured.shape}")

## 6. Merge Power and Weather Data

In [ ]:
def merge_weather_data(power_df, weather_dict):
    """Merge weather data with power generation data"""
    merged_df = power_df.copy()
    
    for weather_type, weather_df in weather_dict.items():
        print(f"Merging {weather_type} data...")
        
        # Merge on Time
        merged_df = pd.merge(merged_df, weather_df, on='Time', how='left')
        
        print(f"  Merged shape: {merged_df.shape}")
        
        # Check missing values after merge
        weather_cols = [col for col in weather_df.columns if col != 'Time']
        missing_after_merge = merged_df[weather_cols].isnull().sum().sum()
        print(f"  Missing values after merge: {missing_after_merge}")
    
    return merged_df

# Merge weather data
print("Merging weather data with power generation data...")
merged_data = merge_weather_data(power_data_featured, resampled_weather)
print(f"\nFinal merged data shape: {merged_data.shape}")

## 7. Handle Missing Values

In [ ]:
# Check missing values in merged data
print("=== MISSING VALUES IN MERGED DATA ===")
missing_summary = merged_data.isnull().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)
print(missing_summary)

# Calculate missing percentage
missing_pct = (missing_summary / len(merged_data)) * 100
print("\nMissing percentages:")
print(missing_pct)

# Handle missing values
def handle_missing_values(df, strategy='forward_fill'):
    """Handle missing values in the dataset"""
    df = df.copy()
    
    # Separate numeric and categorical columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Remove target variable from imputation
    if 'generation(kWh)' in numeric_cols:
        numeric_cols.remove('generation(kWh)')
    
    print(f"Handling missing values for {len(numeric_cols)} numeric columns...")
    
    if strategy == 'forward_fill':
        # Forward fill then backward fill
        df[numeric_cols] = df[numeric_cols].fillna(method='ffill').fillna(method='bfill')
    
    elif strategy == 'interpolate':
        # Linear interpolation
        df[numeric_cols] = df[numeric_cols].interpolate(method='linear')
        # Fill any remaining NaNs
        df[numeric_cols] = df[numeric_cols].fillna(method='ffill').fillna(method='bfill')
    
    elif strategy == 'knn':
        # KNN imputation
        imputer = KNNImputer(n_neighbors=5)
        df[numeric_cols] = imputer.fit_transform(df[numeric_cols])
    
    return df

# Apply missing value handling
print("\nApplying missing value handling...")
cleaned_data = handle_missing_values(merged_data, strategy='interpolate')

# Check if missing values are handled
remaining_missing = cleaned_data.isnull().sum().sum()
print(f"Remaining missing values: {remaining_missing}")

## 8. Create Lag and Rolling Features

In [ ]:
# Sort data by station and time for proper lag feature creation
cleaned_data = cleaned_data.sort_values(['station', 'Time']).reset_index(drop=True)

# Create lag and rolling features for each station separately
print("Creating lag and rolling features for each station...")
featured_data_list = []

for station in cleaned_data['station'].unique():
    station_data = cleaned_data[cleaned_data['station'] == station].copy()
    
    # Create lag features
    station_data = create_lag_features(station_data, 'generation(kWh)', lags=[1, 2, 4, 24, 48])
    
    # Create rolling features
    station_data = create_rolling_features(station_data, 'generation(kWh)', windows=[4, 12, 24, 48])
    
    # Add weather lag features (for key weather variables)
    if 'Irradiance_mean' in station_data.columns:
        station_data = create_lag_features(station_data, 'Irradiance_mean', lags=[1, 2, 4])
    
    if 'Temperature_mean' in station_data.columns:
        station_data = create_lag_features(station_data, 'Temperature_mean', lags=[1, 2, 4])
    
    featured_data_list.append(station_data)
    print(f"Processed {station}: {station_data.shape}")

# Combine all station data
final_featured_data = pd.concat(featured_data_list, ignore_index=True)
print(f"\nFinal featured data shape: {final_featured_data.shape}")

## 9. Feature Selection and Scaling

In [ ]:
# Remove rows with NaN in target variable
final_featured_data = final_featured_data.dropna(subset=['generation(kWh)']).reset_index(drop=True)

# Identify feature columns (exclude target, time, and station)
exclude_cols = ['Time', 'generation(kWh)', 'power(W)', 'station']
feature_cols = [col for col in final_featured_data.columns if col not in exclude_cols]

print(f"Total features: {len(feature_cols)}")
print(f"Sample features: {feature_cols[:10]}")

# Check for infinite values
inf_cols = []
for col in feature_cols:
    if np.isinf(final_featured_data[col]).any():
        inf_cols.append(col)

if inf_cols:
    print(f"\nColumns with infinite values: {inf_cols}")
    # Replace infinite values with NaN and then fill
    final_featured_data[inf_cols] = final_featured_data[inf_cols].replace([np.inf, -np.inf], np.nan)
    final_featured_data[inf_cols] = final_featured_data[inf_cols].fillna(method='ffill').fillna(0)

# Remove any remaining NaN values in features
final_featured_data[feature_cols] = final_featured_data[feature_cols].fillna(0)

print(f"\nFinal dataset shape: {final_featured_data.shape}")
print(f"Missing values in features: {final_featured_data[feature_cols].isnull().sum().sum()}")
print(f"Missing values in target: {final_featured_data['generation(kWh)'].isnull().sum()}")

## 10. Data Scaling and Normalization

In [ ]:
# Prepare data for scaling
X = final_featured_data[feature_cols].copy()
y = final_featured_data['generation(kWh)'].copy()

# Use RobustScaler to handle outliers
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=feature_cols, index=X.index)

print(f"Scaled features shape: {X_scaled.shape}")
print(f"Target variable shape: {y.shape}")
print(f"\nTarget variable statistics:")
print(y.describe())

## 11. Save Processed Data

In [ ]:
# Create final dataset with metadata
processed_data = final_featured_data.copy()
processed_data[feature_cols] = X_scaled

# Save processed data
processed_data.to_csv(os.path.join(OUTPUT_DIR, 'processed_solar_data.csv'), index=False)
print(f"Saved processed data to {OUTPUT_DIR}processed_solar_data.csv")

# Save feature names
feature_info = {
    'feature_columns': feature_cols,
    'target_column': 'generation(kWh)',
    'metadata_columns': ['Time', 'station'],
    'total_features': len(feature_cols),
    'total_samples': len(processed_data),
    'stations': list(processed_data['station'].unique()),
    'date_range': [str(processed_data['Time'].min()), str(processed_data['Time'].max())]
}

import json
with open(os.path.join(OUTPUT_DIR, 'feature_info.json'), 'w') as f:
    json.dump(feature_info, f, indent=2)

# Save scaler
import joblib
joblib.dump(scaler, os.path.join(OUTPUT_DIR, 'feature_scaler.pkl'))
print(f"Saved feature scaler to {OUTPUT_DIR}feature_scaler.pkl")

print(f"\n=== PREPROCESSING COMPLETE ===")
print(f"Final dataset: {processed_data.shape}")
print(f"Features: {len(feature_cols)}")
print(f"Stations: {len(processed_data['station'].unique())}")
print(f"Date range: {processed_data['Time'].min()} to {processed_data['Time'].max()}")
print(f"Target range: {y.min():.4f} to {y.max():.4f} kWh")